# Anac flight delay prediction

## Data Understand

In [1]:
import pandas as pd
import requests
from urllib.parse import quote

BASE_URL = (
    "https://sistemas.anac.gov.br/dadosabertos/"
    "Voos%20e%20opera%C3%A7%C3%B5es%20a%C3%A9reas/"
    "Voo%20Regular%20Ativo%20%28VRA%29/"
)

MONTHS = {
    1: "Janeiro",
    2: "Fevereiro",
    3: "Março",
    4: "Abril",
    5: "Maio",
    6: "Junho",
    7: "Julho",
    8: "Agosto",
    9: "Setembro",
    10: "Outubro",
    11: "Novembro",
    12: "Dezembro",
}

YEARS = [2018]

dfs = []

def file_exists(url):
    try:
        r = requests.head(url, timeout=10)
        return r.status_code == 200
    except requests.RequestException:
        return False

for year in YEARS:
    for month_num, month_name in MONTHS.items():

        folder = f"{month_num:02d} - {month_name}"
        filename = f"VRA_{year}{month_num}.csv"

        url = f"{BASE_URL}{year}/{quote(folder)}/{filename}"

        if not file_exists(url):
            print(f"❌ Missing: {year}-{month_num:02d}")
            continue

        print(f"✔️ Downloading {year}-{month_num:02d}")

        df = pd.read_csv(
            url,
            sep=";",
            encoding="utf-8",
            skiprows=1,
            low_memory=False
        )

        df["year"] = year
        df["month"] = month_num

        dfs.append(df)

# =========================
# FINAL MERGE
# =========================

df_final = pd.concat(dfs, ignore_index=True)
print("✅ All available VRA data merged")
print(df_final.shape)


✔️ Downloading 2018-01
✔️ Downloading 2018-02
✔️ Downloading 2018-03
✔️ Downloading 2018-04
✔️ Downloading 2018-05
✔️ Downloading 2018-06
✔️ Downloading 2018-07
✔️ Downloading 2018-08
✔️ Downloading 2018-09
✔️ Downloading 2018-10
✔️ Downloading 2018-11
✔️ Downloading 2018-12
✅ All available VRA data merged
(1035169, 14)


In [2]:
df_final

,ICAO Empresa Aérea,Número Voo,Código Autorização (DI),Código Tipo Linha,ICAO Aeródromo Origem,ICAO Aeródromo Destino,Partida Prevista,Partida Real,Chegada Prevista,Chegada Real,Situação Voo,Código Justificativa,year,month
0,AFR,454,0,I,LFPG,SBGR,01/01/2018 20:20,NaN,02/01/2018 08:20,NaN,REALIZADO,NaN,2018,1
1,ARG,1248,0,I,SABE,SBGR,01/01/2018 19:30,NaN,01/01/2018 23:15,NaN,REALIZADO,NaN,2018,1
2,ARG,1251,1,I,SBGL,SABE,NaN,01/01/2018 19:53,NaN,01/01/2018 22:10,REALIZADO,NaN,2018,1
3,ARG,1254,1,I,SABE,SBGL,NaN,01/01/2018 11:02,NaN,01/01/2018 14:29,REALIZADO,NaN,2018,1
4,ARG,1278,0,I,SABE,SBFL,01/01/2018 17:55,NaN,01/01/2018 21:05,NaN,REALIZADO,NaN,2018,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035164,AZU,4040,0,N,SBCF,SBBR,31/12/2018 07:45,31/12/2018 08:07,31/12/2018 09:10,31/12/2018 09:26,REALIZADO,NaN,2018,12
1035165,AZU,4034,0,N,SBVT,SBKP,31/12/2018 20:20,31/12/2018 20:20,31/12/2018 22:00,31/12/2018 22:00,REALIZADO,NaN,2018,12
1035166,AZU,4033,0,N,SBRJ,SBKP,31/12/2018 16:35,31/12/2018 16:29,31/12/2018 17:45,31/12/2018 17:28,REALIZADO,NaN,2018,12
1035167,AZU,4029,0,N,SBRJ,SBKP,31/12/2018 07:55,31/12/2018 07:46,31/12/2018 09:05,31/12/2018 08:49,REALIZADO,NaN,2018,12


In [3]:
df_final['year'].unique()

array([2018])

In [4]:
anac_df = df_final.copy()

In [5]:
anac_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035169 entries, 0 to 1035168
Data columns (total 14 columns):
 #   Column                   Non-Null Count    Dtype 
---  ------                   --------------    ----- 
 0   ICAO Empresa Aérea       1035169 non-null  object
 1   Número Voo               1035169 non-null  object
 2   Código Autorização (DI)  1035045 non-null  object
 3   Código Tipo Linha        1034579 non-null  object
 4   ICAO Aeródromo Origem    1035169 non-null  object
 5   ICAO Aeródromo Destino   1035169 non-null  object
 6   Partida Prevista         1023143 non-null  object
 7   Partida Real             881971 non-null   object
 8   Chegada Prevista         1023143 non-null  object
 9   Chegada Real             881976 non-null   object
 10  Situação Voo             1035169 non-null  object
 11  Código Justificativa     267992 non-null   object
 12  year                     1035169 non-null  int64 
 13  month                    1035169 non-null  int64 
dtypes:

In [6]:
df_final.columns.to_list()

['ICAO Empresa Aérea',
 'Número Voo',
 'Código Autorização (DI)',
 'Código Tipo Linha',
 'ICAO Aeródromo Origem',
 'ICAO Aeródromo Destino',
 'Partida Prevista',
 'Partida Real',
 'Chegada Prevista',
 'Chegada Real',
 'Situação Voo',
 'Código Justificativa',
 'year',
 'month']

In [7]:
anac_df.isna().sum()

,0
ICAO Empresa Aérea,0
Número Voo,0
Código Autorização (DI),124
Código Tipo Linha,590
ICAO Aeródromo Origem,0
ICAO Aeródromo Destino,0
Partida Prevista,12026
Partida Real,153198
Chegada Prevista,12026
Chegada Real,153193


##Data Preparation

In [16]:
anac_df.isna().sum()

,0
ICAO Empresa Aérea,0
Número Voo,0
ICAO Aeródromo Origem,0
ICAO Aeródromo Destino,0
Partida Prevista,11986
Partida Real,114397
Situação Voo,0
year,0
month,0


In [9]:
anac_df = anac_df.drop(columns=[
    'Código Autorização (DI)',
    'Código Tipo Linha',
    'Chegada Prevista',
    'Chegada Real',
    'Código Justificativa'
])


In [10]:
anac_df = anac_df[anac_df['Situação Voo']=='REALIZADO']

In [14]:
anac_df

,ICAO Empresa Aérea,Número Voo,ICAO Aeródromo Origem,ICAO Aeródromo Destino,Partida Prevista,Partida Real,Situação Voo,year,month
0,AFR,454,LFPG,SBGR,01/01/2018 20:20,NaN,REALIZADO,2018,1
1,ARG,1248,SABE,SBGR,01/01/2018 19:30,NaN,REALIZADO,2018,1
2,ARG,1251,SBGL,SABE,NaN,01/01/2018 19:53,REALIZADO,2018,1
3,ARG,1254,SABE,SBGL,NaN,01/01/2018 11:02,REALIZADO,2018,1
4,ARG,1278,SABE,SBFL,01/01/2018 17:55,NaN,REALIZADO,2018,1
...,...,...,...,...,...,...,...,...,...
1035164,AZU,4040,SBCF,SBBR,31/12/2018 07:45,31/12/2018 08:07,REALIZADO,2018,12
1035165,AZU,4034,SBVT,SBKP,31/12/2018 20:20,31/12/2018 20:20,REALIZADO,2018,12
1035166,AZU,4033,SBRJ,SBKP,31/12/2018 16:35,31/12/2018 16:29,REALIZADO,2018,12
1035167,AZU,4029,SBRJ,SBKP,31/12/2018 07:55,31/12/2018 07:46,REALIZADO,2018,12


In [18]:
anac_df = anac_df.dropna()

In [19]:
anac_df

,ICAO Empresa Aérea,Número Voo,ICAO Aeródromo Origem,ICAO Aeródromo Destino,Partida Prevista,Partida Real,Situação Voo,year,month
7,AVA,85,SKBO,SBGR,01/01/2018 01:04,01/01/2018 01:06,REALIZADO,2018,1
11,AZU,2404,SBRF,SBMO,01/01/2018 13:50,01/01/2018 13:44,REALIZADO,2018,1
13,AZU,2428,SBCF,SBBR,01/01/2018 17:30,01/01/2018 17:25,REALIZADO,2018,1
17,AZU,2537,SBCF,SBPA,01/01/2018 21:35,01/01/2018 21:52,REALIZADO,2018,1
19,AZU,2596,SBCY,SBPV,01/01/2018 12:15,01/01/2018 11:59,REALIZADO,2018,1
...,...,...,...,...,...,...,...,...,...
1035164,AZU,4040,SBCF,SBBR,31/12/2018 07:45,31/12/2018 08:07,REALIZADO,2018,12
1035165,AZU,4034,SBVT,SBKP,31/12/2018 20:20,31/12/2018 20:20,REALIZADO,2018,12
1035166,AZU,4033,SBRJ,SBKP,31/12/2018 16:35,31/12/2018 16:29,REALIZADO,2018,12
1035167,AZU,4029,SBRJ,SBKP,31/12/2018 07:55,31/12/2018 07:46,REALIZADO,2018,12


In [20]:
anac_df.isna().sum()

,0
ICAO Empresa Aérea,0
Número Voo,0
ICAO Aeródromo Origem,0
ICAO Aeródromo Destino,0
Partida Prevista,0
Partida Real,0
Situação Voo,0
year,0
month,0


In [29]:
anac_df = anac_df.copy()

category_cols = [
    'ICAO Empresa Aérea',
    'Número Voo',
    'ICAO Aeródromo Origem',
    'ICAO Aeródromo Destino',
    'Situação Voo'
]

anac_df[category_cols] = anac_df[category_cols].astype('category')


datetime_cols = ['Partida Prevista',
            'Partida Real']


In [31]:
anac_df[category_cols] = anac_df[category_cols].astype('category')

In [32]:
anac_df[datetime_cols] = anac_df[datetime_cols].apply(
    pd.to_datetime,
    dayfirst=True,
    errors='coerce'
)

In [34]:
anac_df['year'] = anac_df['year'].astype('int16')
anac_df['month'] = anac_df['month'].astype('int8')

In [48]:
anac_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 862919 entries, 7 to 1035168
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   ICAO Empresa Aérea      862919 non-null  category      
 1   Número Voo              862919 non-null  category      
 2   ICAO Aeródromo Origem   862919 non-null  category      
 3   ICAO Aeródromo Destino  862919 non-null  category      
 4   Partida Prevista        862919 non-null  datetime64[ns]
 5   Partida Real            862919 non-null  datetime64[ns]
 6   Situação Voo            862919 non-null  category      
 7   year                    862919 non-null  int16         
 8   month                   862919 non-null  int8          
 9   delay_minutes           862919 non-null  float64       
 10  is_delayed              862919 non-null  int8          
 11  day_of_week             862919 non-null  int8          
 12  week_of_year            862919 non

In [36]:
anac_df['delay_minutes'] = (
    anac_df['Partida Real'] - anac_df['Partida Prevista']
).dt.total_seconds() / 60


In [37]:
anac_df['is_delayed'] = (anac_df['delay_minutes'] > 15).astype('int8')

In [42]:
anac_df['day_of_week'] = (
    anac_df['Partida Prevista']
    .dt.dayofweek
    .astype('int8')
)


In [43]:
anac_df['week_of_year'] = (
    anac_df['Partida Prevista']
    .dt.isocalendar()
    .week
    .astype('int8')
)


In [44]:
anac_df['week_of_month'] = (
    ((anac_df['Partida Prevista'].dt.day - 1) // 7 + 1)
    .astype('int8')
)


In [45]:
anac_df['hour'] = (
    anac_df['Partida Prevista']
    .dt.hour
    .astype('int8')
)


In [46]:
anac_df['is_weekend'] = (
    anac_df['day_of_week'] >= 5
).astype('int8')


In [49]:
anac_df['day_of_year'] = (
    anac_df['Partida Prevista']
    .dt.dayofyear
    .astype('int16')
)


In [58]:
anac_df

,ICAO Empresa Aérea,Número Voo,ICAO Aeródromo Origem,ICAO Aeródromo Destino,year,month,is_delayed,day_of_week,week_of_year,week_of_month,hour,is_weekend,day_of_year
7,AVA,85,SKBO,SBGR,2018,1,0,0,1,1,1,0,1
11,AZU,2404,SBRF,SBMO,2018,1,0,0,1,1,13,0,1
13,AZU,2428,SBCF,SBBR,2018,1,0,0,1,1,17,0,1
17,AZU,2537,SBCF,SBPA,2018,1,1,0,1,1,21,0,1
19,AZU,2596,SBCY,SBPV,2018,1,0,0,1,1,12,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035164,AZU,4040,SBCF,SBBR,2018,12,1,0,1,5,7,0,365
1035165,AZU,4034,SBVT,SBKP,2018,12,0,0,1,5,20,0,365
1035166,AZU,4033,SBRJ,SBKP,2018,12,0,0,1,5,16,0,365
1035167,AZU,4029,SBRJ,SBKP,2018,12,0,0,1,5,7,0,365


In [57]:
cols_to_drop = [
    'Partida Prevista',
    'Partida Real',
    'delay_minutes',
    'Situação Voo'
]

anac_df = anac_df.drop(columns=cols_to_drop)


In [59]:
anac_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 862919 entries, 7 to 1035168
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   ICAO Empresa Aérea      862919 non-null  category
 1   Número Voo              862919 non-null  category
 2   ICAO Aeródromo Origem   862919 non-null  category
 3   ICAO Aeródromo Destino  862919 non-null  category
 4   year                    862919 non-null  int16   
 5   month                   862919 non-null  int8    
 6   is_delayed              862919 non-null  int8    
 7   day_of_week             862919 non-null  int8    
 8   week_of_year            862919 non-null  int8    
 9   week_of_month           862919 non-null  int8    
 10  hour                    862919 non-null  int8    
 11  is_weekend              862919 non-null  int8    
 12  day_of_year             862919 non-null  int16   
dtypes: category(4), int16(2), int8(7)
memory usage: 21.8 MB


In [41]:
anac_df['is_delayed'].value_counts(normalize=True)


,proportion
is_delayed,
0,0.830933
1,0.169067
